In [2]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import os

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
cyberball_raw_dir = Path('Physiological (ECG, EDA, RSP)/')
cyberball_interim_dir = Path('interim/')

In [6]:
for txt in tqdm(list(cyberball_raw_dir.glob("*.txt"))):
    # print(txt)
    try:
        _, _, patient_id, paradigm, signal = txt.name.split(".")[0].split("_")
        df_sig = pd.read_csv(
            txt, skiprows=3, sep=" ", names=["timestamp", signal, "trigger", "dunno"]
        )
        t_start = pd.to_datetime(open(txt).readlines()[1].strip()).tz_localize(
            "Europe/Brussels"
        )
        df_sig.timestamp = t_start + pd.to_timedelta(df_sig.timestamp, unit='ms')
        df_sig = df_sig.set_index('timestamp').iloc[:, :2]
        df_sig[signal] = df_sig[signal].astype(np.float32)
        # df_sig["trigger"] = df_sig["trigger"].astype(np.int32)
        save_dir = cyberball_interim_dir.joinpath(patient_id)
        if not save_dir.is_dir():
            os.makedirs(save_dir)
        df_sig.to_parquet(save_dir.joinpath(f'{signal}_{paradigm}.parquet'))
    except Exception as e:
        print("[E]", txt.name, e)
    # /1024_ppt_8_cybb_SCL.txt

0it [00:00, ?it/s]